In [1]:
import pandas as pd
import pymysql
import json

# =============================================================================
# 1. CSV 파일 로드 - 사원명 → ID 매핑 딕셔너리 생성
# =============================================================================
csv_path = "인사정보_부서코드추가.csv"
df_emp = pd.read_csv(csv_path, encoding='utf-8')

# 사원명 → ID 딕셔너리
name_to_id = dict(zip(df_emp['사원명'], df_emp['ID']))
print(f"직원 매핑 로드 완료: {len(name_to_id)}명")

# =============================================================================
# 2. DB 연결 및 comments 테이블 조회
# =============================================================================
conn = pymysql.connect(
    host='localhost',
    user='root',      # 실제 DB 사용자명으로 변경
    password='1234',  # 실제 DB 비밀번호로 변경
    database='any_approval',
    charset='utf8mb4'
)

query = "SELECT source_id, source_document_id, created_at, updated_at, writer, message FROM comments"
df_comments = pd.read_sql(query, conn)
conn.close()

print(f"comments 데이터 로드 완료: {len(df_comments)}건")

# =============================================================================
# 3. 메시지 이스케이프 처리 함수
# =============================================================================
def escape_message(msg):
    if msg is None:
        return ""
    # 백슬래시 먼저 처리 (순서 중요)
    msg = msg.replace('\\', '\\\\')
    # 따옴표 이스케이프
    msg = msg.replace('"', '\\"')
    # 줄바꿈 이스케이프
    msg = msg.replace('\n', '\\n')
    msg = msg.replace('\r', '\\r')
    # 탭 이스케이프
    msg = msg.replace('\t', '\\t')
    return msg

# =============================================================================
# 4. addDocumentComment 형식으로 변환
# =============================================================================
result_lines = []

for idx, row in df_comments.iterrows():
    writer_name = row['writer']
    
    # 사원명으로 emailId 찾기
    email_id = name_to_id.get(writer_name, None)
    if email_id is None:
        print(f"[경고] 매핑 없음(퇴사자): {writer_name} (row {idx}) → master 처리")
        email_id = "master"
    
    # 메시지 이스케이프 처리
    escaped_message = escape_message(row['message'])
    
    # JSON 형식 문자열 직접 구성 (줄바꿈 없이 한 줄로)
    cmd = (
        f'addDocumentComment {{'
        f'"sourceId":"{row["source_id"]}",'
        f'"createdAt":"{row["created_at"]}",'
        f'"writer":{{"emailId":"{email_id}"}},'
        f'"message":"{escaped_message}",'
        f'"sourceDocumentId":"{row["source_document_id"]}",'
        f'"updatedAt":"{row["updated_at"]}"'
        f'}}'
    )
    result_lines.append(cmd)

# =============================================================================
# 5. 결과 파일 저장
# =============================================================================
output_path = "comments_cmds.txt"
with open(output_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(result_lines))

print(f"\n완료! {len(result_lines)}건 저장됨: {output_path}")

# 미리보기
print("\n=== 미리보기 (처음 3건) ===")
for line in result_lines[:3]:
    print(line)
    print()

직원 매핑 로드 완료: 156명
comments 데이터 로드 완료: 5건
[경고] 매핑 없음: 김대일 (row 4)

완료! 5건 저장됨: comments_cmds.txt

=== 미리보기 (처음 3건) ===
addDocumentComment {"sourceId":"01_23279081_01","createdAt":"1720673708000","writer":{"emailId":"jychoi"},"message":"본 건은 MS-SQL 연결작업의 내부처리가 불가하여, 전문업체의 기술지원을 받아 발생하였습니다.\n프로젝트 완료처리를 위한 필수작업으로 CP에는 반영되지 않은 추가비용입니다.","sourceDocumentId":"23279081","updatedAt":"1720673708000"}

addDocumentComment {"sourceId":"01_26322481_01","createdAt":"1760585241000","writer":{"emailId":"kimbr"},"message":"시설장비비를 간접비로 6,300만원 전용하여 사용.","sourceDocumentId":"26322481","updatedAt":"1760585241000"}

addDocumentComment {"sourceId":"02_26322481_01","createdAt":"1760686729000","writer":{"emailId":"kimbr"},"message":"시설장비비를 간접비로 630만원 전용하여 사용(6300만원X)","sourceDocumentId":"26322481","updatedAt":"1760686729000"}



C:\Users\LEEJUHWAN\AppData\Local\Temp\ipykernel_3872\2978849820.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_comments = pd.read_sql(query, conn)
